# Pseudocode

In [ ]:
# Create an webcam to show a video
# Capture the hand
    # We install the mediapipe library
    # We use the holistic and drawing utils in the mediapipe.solutions library
    # We loop through the mp_hand obtained earier to be able to use the hand function
    # We set use the hand and drawing utils to capture the hand
        # We first convert the video to RGB
        # We train the frame
        # Convert it back to BGR
        # We draw using the landmarks obtained from the result and the connections obtained from the drawing_tils

# Obtain the landmarks in a list
    # We first obtain the landmark from hand_landmarks.landmark, we use loop to get the x, y, z and visibility
    # We cnvert it to an array
    # We apply the list function to convert it to a list
# Save the landmarks in a csv file (I'll save it for each class)
    # We will first create a row for the file using list comprehension.
    # We will then save it to our csv file using the csv module (csv.writerow())
    # As each frame in the video runs, it will get the landmarks to be equal to the lenght of the column in the row as a list
    # We will then append the list of the landmarks into the csv file as each frame runs
    # There are 21 landmarks on the human hand
    # Ther will be (x, y, z, visibility) making 84 elements in the list
    # If we add the class names it will make 85 in total.
    
# Train it
    # For training, i will use the random forrest, logistic regression, ridge classifier and gradient boosting classifier.
    # The rest are the normal machine learning training steps
# Predict the class for which the hand posture belongs to.
    # We will first get the saved file from the pre trained model
    # We will now apply it in the main detection video webcam to predict it.
    # Once it has been predicted, we will use the opencv library to show it.

# Import The Necessary Dependences

In [1]:
import mediapipe as mp
import cv2 as cv
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistics = mp.solutions.holistic

# Setting up the cam and doing some detections

In [529]:
cap = cv.VideoCapture(0)

with mp_holistics.Holistic(min_detection_confidence= 0.3, min_tracking_confidence= 0.3) as holistics:
    while cap.isOpened():
        ret, frame = cap.read()

        # convert the color of the image to RGB
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        
        # Render detection by passing it into the model
        results = holistics.process(image)
        
        # Convert it back to BGR
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        
        
        # Draw landmarks (left_hand_landmarks and right_hand_landmarks)
        # left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistics.HAND_CONNECTIONS)
        # right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistics.HAND_CONNECTIONS)


        cv.imshow("Window", image)
        if cv.waitKey(10) & 0xFF ==ord('q'):
            break
cap.release()
cv.destroyAllWindows()

In [97]:
cap.release()
cv.destroyAllWindows()

In [49]:
norm_coords = len(results.right_hand_landmarks.landmark) + len(results.left_hand_landmarks.landmark)
norm_coords

42

# Saving the hand posture to csv

In [499]:
noR = (21 + 21)
noR

42

In [500]:
# list_landmark = ['className']
# for i in range(1, noR+1):
#     list_landmark += [f"x{i}", f"y{i}", f"z{i}", f"v{i}"]

In [3]:
import csv
import os

In [502]:
# with open("train90deg.csv", mode='w', newline= '') as f:
#     csv_writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
#     csv_writer.writerow(list_landmark)

In [83]:
class_name = "Y"

In [84]:
cap = cv.VideoCapture(0)

with mp_holistics.Holistic(min_detection_confidence= 0.3, min_tracking_confidence= 0.3) as holistics:
    while cap.isOpened():
        ret, frame = cap.read()

        # convert the color of the image to RGB
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        
        # Render detection by passing it into the model
        results = holistics.process(image)
        
        # Convert it back to BGR
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        
        
        # Draw landmarks (left_hand_landmarks and right_hand_landmarks)
        # left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistics.HAND_CONNECTIONS)
        # right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistics.HAND_CONNECTIONS)
        

        left_hand = results.left_hand_landmarks
        right_hand = results.right_hand_landmarks
        
        try:
            if left_hand == None:
                left_row = list(np.zeros(21*4))
                right_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in right_hand.landmark]).flatten())
            elif right_hand == None:
                right_row = list(np.zeros(21*4))
                left_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in left_hand.landmark]).flatten())
            else:
                left_hand = left_hand.landmarks
                right_hand = right_hand.landmarks

                left_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in left_hand]).flatten())
                right_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in right_hand]).flatten())


            # Lets add the both landmarks together

            # Adding the two list together
            landm = left_row + right_row

            # Inserting the class name at the first row
            landm.insert(0, class_name)

            # Putting each and every parameter the csv file
            with open("train90deg.csv", mode='a', newline= '') as f:
                csv_writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                csv_writer.writerow(landm)
        except:
            pass

        


        cv.imshow("Window", image)
        if cv.waitKey(10) & 0xFF ==ord('q'):
            break
cap.release()
cv.destroyAllWindows()

# Splitting into train and test

In [85]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [86]:
df = pd.read_csv("train90deg.csv")
df

,className,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z40,v40,x41,y41,z41,v41,x42,y42,z42,v42
0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.050040,0.0,0.289515,0.574584,-0.036493,0.0,0.284205,0.612984,-0.013090,0.0
1,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.043806,0.0,0.290689,0.571865,-0.029985,0.0,0.285241,0.609139,-0.006407,0.0
2,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.036508,0.0,0.292213,0.566322,-0.021161,0.0,0.286835,0.604519,0.001996,0.0
3,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.042254,0.0,0.293157,0.565786,-0.027396,0.0,0.286887,0.603424,-0.003332,0.0
4,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.044045,0.0,0.294258,0.566601,-0.029204,0.0,0.288807,0.601601,-0.005547,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12758,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.013259,0.0,0.247453,0.238498,-0.013685,0.0,0.255262,0.199457,-0.009992,0.0
12759,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.012884,0.0,0.238226,0.246538,-0.014225,0.0,0.243492,0.207259,-0.010457,0.0
12760,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.052837,0.0,0.213773,0.253915,-0.066809,0.0,0.220172,0.208587,-0.069401,0.0
12761,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.007587,0.0,0.184120,0.255340,-0.011548,0.0,0.186648,0.215338,-0.008996,0.0


In [87]:
df.head()

,className,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z40,v40,x41,y41,z41,v41,x42,y42,z42,v42
0,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.050040,0.0,0.289515,0.574584,-0.036493,0.0,0.284205,0.612984,-0.013090,0.0
1,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.043806,0.0,0.290689,0.571865,-0.029985,0.0,0.285241,0.609139,-0.006407,0.0
2,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.036508,0.0,0.292213,0.566322,-0.021161,0.0,0.286835,0.604519,0.001996,0.0
3,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.042254,0.0,0.293157,0.565786,-0.027396,0.0,0.286887,0.603424,-0.003332,0.0
4,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.044045,0.0,0.294258,0.566601,-0.029204,0.0,0.288807,0.601601,-0.005547,0.0


In [88]:
df.tail()

,className,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z40,v40,x41,y41,z41,v41,x42,y42,z42,v42
12758,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.013259,0.0,0.247453,0.238498,-0.013685,0.0,0.255262,0.199457,-0.009992,0.0
12759,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.012884,0.0,0.238226,0.246538,-0.014225,0.0,0.243492,0.207259,-0.010457,0.0
12760,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.052837,0.0,0.213773,0.253915,-0.066809,0.0,0.220172,0.208587,-0.069401,0.0
12761,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.007587,0.0,0.184120,0.255340,-0.011548,0.0,0.186648,0.215338,-0.008996,0.0
12762,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.011601,0.0,0.170567,0.265522,-0.012540,0.0,0.173416,0.225322,-0.008700,0.0


In [89]:
df[df.className == 'B']

,className,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z40,v40,x41,y41,z41,v41,x42,y42,z42,v42
368,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.030710,0.0,0.332362,0.262070,-0.037795,0.0,0.343787,0.202048,-0.043513,0.0
369,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.028111,0.0,0.336610,0.259622,-0.036436,0.0,0.346310,0.200025,-0.043199,0.0
370,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.029457,0.0,0.339799,0.257688,-0.037502,0.0,0.350187,0.198224,-0.043978,0.0
371,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.033106,0.0,0.336552,0.255973,-0.040704,0.0,0.345745,0.196668,-0.046909,0.0
372,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.032036,0.0,0.333922,0.258926,-0.039552,0.0,0.343209,0.199795,-0.045867,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.048243,0.0,0.209806,0.403228,-0.055516,0.0,0.208048,0.337197,-0.059899,0.0
784,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.048297,0.0,0.216594,0.407542,-0.057917,0.0,0.215747,0.336364,-0.064991,0.0
785,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.065929,0.0,0.221434,0.402859,-0.076274,0.0,0.221966,0.338224,-0.082037,0.0
786,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.040961,0.0,0.220599,0.407377,-0.050842,0.0,0.218899,0.339517,-0.058005,0.0


In [90]:
X = df.drop("className", axis = 1)
y = df.className

In [91]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [92]:
osx_train, osy_train = SMOTE().fit_resample(x_train, y_train)

In [93]:
osy_train

0        G
1        S
2        K
3        I
4        Y
        ..
15895    Z
15896    Z
15897    Z
15898    Z
15899    Z
Name: className, Length: 15900, dtype: object

In [94]:
pipelines = {
    'linearReg': make_pipeline(StandardScaler(), LogisticRegression()),
    'randomForrest' : make_pipeline(StandardScaler(), RandomForestClassifier()),
    'decisionTree' : make_pipeline(StandardScaler(), DecisionTreeClassifier()),
    'ridgeReg' : make_pipeline(StandardScaler(), RidgeClassifier()),
    'gradBoosting' : make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

# Training process

In [95]:
model_fit = {}
for algorithm, pipeline in pipelines.items():
    model_fit[algorithm] = pipeline.fit(osx_train, osy_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [96]:
y_test = model_fit['randomForrest'].predict(x_test)
y_test

array(['Z', 'K', 'N', ..., 'T', 'I', 'D'], dtype=object)

In [97]:
from sklearn.metrics import accuracy_score

In [98]:
result = {}
for algorithm, pipeline in pipelines.items():
    y_pred = pipeline.predict(x_test)
    result[algorithm] = accuracy_score(y_test, y_pred)

In [99]:
result

{'linearReg': 0.9949858978376684,
 'randomForrest': 1.0,
 'decisionTree': 0.970542149796302,
 'ridgeReg': 0.9902851770604826,
 'gradBoosting': 0.9924788467565027}

# Save the trained model

In [100]:
import pickle

In [101]:
with open("ssLang90deg.pkl", mode = 'wb') as f:
    pickle.dump(model_fit['randomForrest'], f)

# Load the model for detection

In [102]:
with open("ssLang90deg.pkl", mode = "rb") as f:
    model = pickle.load(f)

In [ ]:
cap = cv.VideoCapture(0)

with mp_holistics.Holistic(min_detection_confidence= 0.3, min_tracking_confidence= 0.3) as holistics:
    while cap.isOpened():
        ret, frame = cap.read()

        # convert the color of the image to RGB
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        
        # Render detection by passing it into the model
        results = holistics.process(image)
        
        # Convert it back to BGR
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        
        
        # Draw landmarks (left_hand_landmarks and right_hand_landmarks)
        # left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistics.HAND_CONNECTIONS)
        # right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistics.HAND_CONNECTIONS)
        

        left_hand = results.left_hand_landmarks
        right_hand = results.right_hand_landmarks
        
        # Draw the rectangle
        cv.rectangle(image, (225, 0), (375, 20), (255, 255, 255),cv.FILLED)
        # Show the class
        cv.putText(image, 'CLASS',
                      (320, 12), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
        # Show the probability class
        cv.putText(image, 'PROB',
                      (240, 12), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
        
        try:
            if left_hand == None:
                left_row = list(np.zeros(21*4))
                right_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in right_hand.landmark]).flatten())
            elif right_hand == None:
                right_row = list(np.zeros(21*4))
                left_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in left_hand.landmark]).flatten())
            else:
                left_hand = left_hand.landmarks
                right_hand = right_hand.landmarks

                left_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in left_hand]).flatten())
                right_row = list(np.array([[landmarks.x, landmarks.y, landmarks.z, landmarks.visibility] for landmarks in right_hand]).flatten())


            # Lets add the both landmarks together

            # Adding the two list together
            landm = left_row + right_row
            
            
            X = pd.DataFrame([landm])
            lang_prediction = model.predict(X)[0]
            lang_probability = model.predict_proba(X)[0]
            probability = round(lang_probability[np.argmax(lang_probability)], 2)
            
            if probability > 0.3:
                # Display Class
                cv.putText(image, lang_prediction.split(' ')[0],
                          (320, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)

                # Display Probability
                cv.putText(image, str(probability),
                          (235, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            
        except:
            pass

        
        cv.imshow("Window", image)
        if cv.waitKey(10) & 0xFF ==ord('q'):
            break
cap.release()
cv.destroyAllWindows()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid f

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid f

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid f

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid f

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid f

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid f

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid f

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid f

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid f